In [1]:
#-*- coding: utf-8 -*-
# 프로그램 목적 : KorPatBERT 와 Keras를 활용하여, 3개의 클래스로 문장을 분류하는 테스트 프로그램
# 사용되어진 데이터 셋은 label, sentence 두개의 컬럼으로 구성되어 있으며,
# 클래스별 300개씩 총 900개의 데이터로 이루어져 있다.
# label은 특허 CPC코드 섹션 "A", "B", "C" 3개의 클래스로 구성되어 있고, 
# sentence 는 특허기술에 대한 문장으로 이루어져 있다.

# 필요 라이브러리 임포트
import tensorflow as tf
import bert
import os
import mecab
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tqdm import tqdm
from korpat_tokenizer import Tokenizer

In [2]:
import tensorflow as tf

print(tf.__version__)

2.7.0


In [18]:
pip install --upgrade tensorflow==2.2.0

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install bert


Note: you may need to restart the kernel to use updated packages.


In [10]:
# 필요 환경변수 설정
os.environ['TF_KERAS'] = '1'    # Keras Tensorflow 설정
config_path     = "./pretrained/korpat_bert_config.json" # KorpatBert Config 파일 경로
vocab_path      = "./pretrained/korpat_vocab.txt"        # KorpatTokenizer Vocabulary 파일 경로
checkpoint_path = "./pretrained/model.ckpt-381250"       # KorpatBert 모델파일 경로
pretreind_model_dir = "./pretrained/"                    # KorpatBert 모델 디렉토리 경로
dataset_path = "./lm_test_data.tsv"                      # 사용할 데이터셋 경로
save_model_path = "./korpat_bert_test_model.h5"          # 학습완료 모델 저장 경로

MAX_SEQ_LEN = 256 # 학습 최대 토큰 갯수
BATCH_SIZE = 9    # 학습 배치 사이즈 기본8
EPOCHS = 5        # 학습 에폭
LR = 0.00003      # 학습률

In [11]:
# 분리 비율에 따른 데이터셋 분리 함수
# 입력 : dataset(분리 대상 데이터셋), split_val(분리 비율)
# 출력 : train_data(분리 데이터셋), dev_data(분리 데이터셋)
def dataset_split(dataset, split_val):
    lengths = int(len(dataset) * split_val)
    train_data = dataset[:lengths]
    dev_data = dataset[lengths:]
    return train_data, dev_data

# 데이터 셋 로드 함수
# 입력 : dataset_url (전체 데이터셋 경로)
# 출력 : train_data(학습데이터) dev_data(검증데이터), test_data(평가데이터)
def dataset_load(dataset_url):
    all_data = pd.read_csv(dataset_url, sep='\t')
    all_data = all_data.sample(frac=1).reset_index(drop=True)
    train_data, test_data = dataset_split(dataset=all_data, split_val=0.9)
    train_data, dev_data = dataset_split(dataset=train_data, split_val=0.9)

    return train_data, dev_data, test_data

# 학습을 위한 데이터 셋 전처리 함수
# 입력 : dataset(전처리 대상 데이터셋)
# 출력 : tokens(토큰화 결과), x_data(입력데이터), y_data(정답데이터)
def preprocessing_dataset(dataset):
    tokens, indices, labels = [], [],[]
    
    # 데이터셋의 문장을 토큰화 및 인코딩 처리하고, 라벨을 One hot 벡터 변환으로 처리한다.
    for label, sentence in tqdm(zip(dataset['label'], dataset['sentence']), desc = "데이터 전처리 진행중"):
        tokens.append(tokenizer.tokenize(sentence))
        ids, _ = tokenizer.encode(sentence, max_len=MAX_SEQ_LEN)
        indices.append(ids)

        if label == "A":
            labels.append([1, 0, 0])
        elif label == "B":
            labels.append([0, 1, 0])
        else:
            labels.append([0, 0, 1])

    x_data = np.array(indices)
    y_data = np.array(labels)
    print("===> 전처리 결과 출력 <===")
    print("===> tokens sample : ", tokens[0])
    print("===> indices sample : ", indices[0])
    print("===> x_data shape : ", x_data.shape)
    print("===> y_data shape : ", y_data.shape)
    
    return tokens, x_data, y_data

In [12]:
# KorPat Tokenier 선언
tokenizer = Tokenizer(vocab_path=vocab_path, cased=True)

# 필요 데이터셋 로드
train_data, dev_data, test_data = dataset_load(dataset_path)

# 모든 데이터셋 전처리
print("===> 학습데이터 샘플 출력 및 전처리 시작 <===")
print(train_data[:10])
train_tokens, train_x, train_y = preprocessing_dataset(train_data)

print("\n\n===> 검증데이터 샘플 출력 및 전처리 시작 <===")
print(dev_data[:10])
dev_tokens, dev_x, dev_y = preprocessing_dataset(dev_data)

print("\n\n===> 평가데이터 샘플 출력 및 전처리 시작 <===")
print(test_data[:10])
test_tokens, test_x, test_y = preprocessing_dataset(test_data)

# 라벨의 크기 정의
label_size = train_y.shape[1]
print("===> 라벨 사이즈 : ", label_size)

===> 학습데이터 샘플 출력 및 전처리 시작 <===
  label                                           sentence
0     B  초음파 세척기 본 발명은 진동소자와 연계되어 진동을 일으키는 진동판의 대면적화를 통...
1     A  미네랄볼을 이용한 산야초 발효촉진 방법 및 산야초 발효추출액 본 발명은 산야초 고유...
2     B  신용카드 매출전표 본 발명은 신용카드 매출전표에 관한 것으로 더욱 상세하게는 세 장...
3     A  탄산칼슘, 식품 첨가용 탄산칼슘 제제 및 식품 음료 등의 식품에 첨가했을 경우에, ...
4     A  복합 발효 하수오 추출물과 한약재 추출물을 이용한 갱년기 증상 개선용 음료의 제조방...
5     C  슬래그의 포밍 억제 방법 및 전로 정련 방법 전로의 하방에 설치한 배재 레이들로 노...
6     B  사진첩 본 고안은 사진을 회전캡으로 회전시키면서 간단하게 보관이나 열람이 가능토록 ...
7     C  표면 처리 금속판, 전지 용기 및 전지 금속판 및 상기 금속판 상에 형성된 니켈-코...
8     A  영양소가 풍부한 기능성 두부 제조방법 본 발명은 영양소가 풍부한 기능성 두부 제조방...
9     C  공배양 몰드, 공배양 방법 및 폐 장기 모사칩 본 발명에 따른 공배양 몰드는 중앙부...


데이터 전처리 진행중: 729it [00:05, 132.94it/s]


===> 전처리 결과 출력 <===
===> tokens sample :  ['[CLS]', '초음파', '세척기', '본', '발명', '은', '진동', '소자', '와', '연계', '되', '어', '진동', '을', '일으키', '는', '진동판', '의', '대', '면적', '화', '를', '통해', '세', '척', '효율', '을', '한층', '향상', '시킬', '수', '있', '는', '초음파', '세척기', '에', '관한', '것', '이', '다', '.', '본', '발명', '은', '하나', '또는', '복수', '개', '의', '단위', '진동자', '와', '대', '면적', '의', '진동판', '을', '가지', '는', '케이스', '의', '조합', '으로', '이루어진', '진동', '모듈', '을', '채택', '하', '여', '진동', '소자', '로부터', '발생', '되', '는', '진동', '파', '가', '다양', '한', '경로', '로', '전달', '될', '수', '있', '도록', '함', '과', '더불', '어', '진동', '의', '세기', '와', '강도', '를', '높일', '수', '있', '도록', '한', '새로운', '형태', '의', '진동', '방식', '을', '구현', '함', '으로써', ',', '적', '은', '수', '의', '진동자', '만', '으로', '도', '세', '척', '능력', '을', '크', '게', '높일', '수', '있', '는', '동시', '에', '제작', '비용', '절감', '에', '따라', '경제', '적', '으로', '도', '유리', '하', '고', ',', '소음', '발생', '문제', '를', '대폭', '개선', '할', '수', '있', '으며', ',', '제작', '및', '조립', '이', '쉬', '##울', '뿐', '만', '아니', '라', 'A', '/', 'S', '는', '물론',

데이터 전처리 진행중: 81it [00:00, 134.99it/s]


===> 전처리 결과 출력 <===
===> tokens sample :  ['[CLS]', '미', '##라', '##클', '프', '##루', '##트', '를', '포함', '하', '는', '스', '테', '비아', '감미', '음료', '에', '대한', '맛', '개선', '스', '테', '비아', '성분', ',', '미', '##라', '##클', '프', '##루', '##트', '성분', '및', '음료', '를', '함유', '하', '는', '조성물', '.', '미', '##라', '##클', '프', '##루', '##트', '를', '포함', '하', '는', '스', '테', '비아', '감미', '음료', '의', '제조', '방법', '.', '[SEP]']
===> indices sample :  [5, 222, 20362, 20540, 122, 20513, 20333, 16, 79, 12, 11, 94, 360, 3623, 11244, 3599, 10, 241, 5044, 1219, 94, 360, 3623, 622, 8, 222, 20362, 20540, 122, 20513, 20333, 622, 44, 3599, 16, 515, 12, 11, 454, 14, 222, 20362, 20540, 122, 20513, 20333, 16, 79, 12, 11, 94, 360, 3623, 11244, 3599, 9, 216, 157, 14, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

데이터 전처리 진행중: 90it [00:00, 119.85it/s]

===> 전처리 결과 출력 <===
===> tokens sample :  ['[CLS]', '방향', '성', '전자', '강판', '및', '방향', '성', '전자', '강판', '의', '제조', '방법', '본', '발명', '의', '일', '양태', '에', '관한', '방향', '성', '전자', '강판', '은', ',', '강판', '과', ',', '강판', '상', '에', '배치', '된', 'Si', '및', 'O', '를', '포함', '하', '는', '중간층', '과', ',', '중간층', '상', '에', '배치', '된', '절연', '피막', '을', '갖', '는', '방향', '성', '전자', '강판', '이', '며', ',', '중간층', '이', '금속', '인', '##화물', '을', '함유', '하', '고', ',', '중간층', '의', '층', '두께', '가', '4', 'nm', '이상', '이', '고', ',', '금속', '인', '##화물', '의', '존재', '량', '이', ',', '중간층', '의', '단면', '에', '있', '어서', '의', '단면', '면적', '률', '로', '1', '내지', '30', '[UNK]', '이', '다', '.', '[SEP]']
===> indices sample :  [5, 280, 106, 403, 3508, 44, 280, 106, 403, 3508, 9, 216, 157, 58, 77, 9, 70, 2239, 10, 439, 280, 106, 403, 3508, 32, 8, 3508, 55, 8, 3508, 17, 10, 394, 30, 1235, 44, 318, 16, 79, 12, 11, 7377, 55, 8, 7377, 17, 10, 394, 30, 626, 4287, 15, 237, 11, 280, 106, 403, 3508, 13, 134, 8, 7377, 13, 412, 53, 1503, 15, 515, 12, 34, 

In [16]:
# 분류 모델 네트워크 정의, ※ MirroredStrategy (분산처리 포함)
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # 입력 레이어 생성
    input_ids  = keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype='int32')
    
    # BERT 언어모델 레이어 생성
    bert_params = bert.params_from_pretrained_ckpt(pretreind_model_dir)
    l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")
    
    # 입력레이어와 BERT 레이어 연결
    bert_output = l_bert(input_ids) 
    print("bert shape", bert_output.shape)
    
    # BERT 출력 중 Context 정보가 담긴, [CLS] 토큰 정보를 추출
    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    
    # 최종 분류를 위한 클래스 개수로 구성된 Dense 레이어를 연결
    outputs = Dense(units=3, activation='softmax')(cls_out) #    outputs = Dense(units=label_size, activation='softmax')(cls_out) 
    
    # 모델 빌딩
    model = keras.Model(inputs=input_ids, outputs=outputs)
    model.build(input_shape=(None, MAX_SEQ_LEN))
    
    # BERT 언어모델의 초기 가중치 로드
    bert.load_stock_weights(l_bert, checkpoint_path)
    
    # 모델 컴파일 과정
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LR),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )
# 모델 요약 출력
model.summary()    

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


TypeError: Exception encountered when calling layer "bert" (type BertModelLayer).

Layer input_spec must be an instance of InputSpec. Got: InputSpec(shape=(None, 256, 768), ndim=3)

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 256), dtype=int32)
  • mask=None
  • training=None

In [8]:
# 모델 학습 시작
history = model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(dev_x, dev_y)
)

NameError: name 'model' is not defined

In [ ]:
# 모델 저장 및 테스트 데이터를 이용한 평가
model.save(save_model_path)
eval_result = model.evaluate(test_x, test_y)

print('\n===> 평가 결과 출력')
print("Accuracy : %.4f" % (eval_result[1]))